In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import Any
from logging import INFO, DEBUG
import json
import os
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from scipy.signal import medfilt
from flwr.common import log, ndarrays_to_parameters
import matplotlib.pyplot as plt
import math

from src.common.client_utils import (
    load_femnist_dataset,
    get_network_generator_cnn as get_network_generator,
    get_device,
    get_model_parameters,
    aggregate_weighted_average,
)


from src.flwr_core import (
    set_all_seeds,
    get_paths,
    decompress_dataset,
    get_flower_client_generator,
    sample_random_clients,
    get_federated_evaluation_function,
    create_iid_partition,
)

from src.estimate import (
    compute_critical_batch,
)

from src.experiments_simulation import (
    run_simulation,
    centralized_experiment,
)

from src.utils import get_centralized_acc_from_hist

PathType = Path | str | None

/home/lcm76/.conda/envs/fl-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-16 22:50:49,257	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


**Global variables**

In [3]:
set_all_seeds()

PATHS = get_paths()

HOME_DIR = PATHS["home_dir"]
DATASET_DIR = PATHS["dataset_dir"]
DATA_DIR = PATHS["data_dir"]
CENTRALIZED_PARTITION = PATHS["centralized_partition"]
CENTRALIZED_MAPPING = PATHS["centralized_mapping"]
FEDERATED_PARTITION = PATHS["federated_partition"]
FEDERATED_IID_PARTITION = PATHS["iid_partition"]

# extract dataset from tar.gz
decompress_dataset(PATHS)

In [4]:
max_clients = 1600
create_iid_partition(PATHS, num_clients=max_clients)

Distributing data to client 0
Distributing data to client 1
Distributing data to client 2
Distributing data to client 3
Distributing data to client 4
Distributing data to client 5
Distributing data to client 6
Distributing data to client 7
Distributing data to client 8
Distributing data to client 9
Distributing data to client 10
Distributing data to client 11
Distributing data to client 12
Distributing data to client 13
Distributing data to client 14
Distributing data to client 15
Distributing data to client 16
Distributing data to client 17
Distributing data to client 18
Distributing data to client 19
Distributing data to client 20
Distributing data to client 21
Distributing data to client 22
Distributing data to client 23
Distributing data to client 24
Distributing data to client 25
Distributing data to client 26
Distributing data to client 27
Distributing data to client 28
Distributing data to client 29
Distributing data to client 30
Distributing data to client 31
Distributing data 

In [5]:
NETWORK_GENERATOR = get_network_generator()
SEED_NET = NETWORK_GENERATOR()
SEED_MODEL_PARAMS = get_model_parameters(SEED_NET)
CID_CLIENT_GENERATOR = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_IID_PARTITION)

In [6]:
# Centralized experiments
centralized_experiment_batch_sizes = [16, 32, 64, 128, 256, 512, 1024]

# Load the centralized dataset using the same function as in FL.
# The centralized mapping folder should be the one used in the FL centralized experiment.
centralized_train_dataset = load_femnist_dataset(data_dir=DATA_DIR,mapping=CENTRALIZED_MAPPING, name="train")
centralized_test_dataset = load_femnist_dataset(data_dir=DATA_DIR, mapping=CENTRALIZED_MAPPING, name="test")

centralized_train_config = {
    "epochs": 10,
    "batch_size": 32,
    "client_learning_rate": 0.01,
    "weight_decay": 0,
    "num_workers": 0,
    "max_batches": 100,
}

centralized_test_config = {
    "batch_size": 32,
    "num_workers": 0,
    "max_batches": 100,
    "target_accuracy": 0.60,
}

In [7]:
# FL experiments
experiment_batch_sizes = [16, 32, 64, 128, 256]
cohort_sizes = [5, 10, 20, 50, 75, 100, 150]


# Federated configuration dictionary
federated_train_config = {
    "epochs": 10,
    "batch_size": 32,
    "client_learning_rate": 0.01,
    "weight_decay": 0,
    "num_workers": 0,
    "max_batches": 100,
}

federated_test_config: dict[str, Any] = {
    "batch_size": 32,
    "num_workers": 0,
    "max_batches": 100,
}

num_rounds = 10
num_total_clients = 500
num_evaluate_clients = 0
num_clients_per_round = 10

initial_parameters = ndarrays_to_parameters(SEED_MODEL_PARAMS)

federated_evaluation_function = get_federated_evaluation_function(
    batch_size=federated_test_config["batch_size"],
    num_workers=federated_test_config["num_workers"],
    model_generator=NETWORK_GENERATOR,
    criterion=nn.CrossEntropyLoss(),
    max_batches=None if "max_batches" not in federated_test_config else federated_test_config["max_batches"],
)

server_learning_rate = 1.0
server_momentum = 0.0
accept_failures = False


CID_CLIENT_GENERATOR = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_IID_PARTITION)

list_of_ids = sample_random_clients(
    num_total_clients, federated_train_config["batch_size"],
    CID_CLIENT_GENERATOR, max_clients=max_clients,
)

federated_client_generator = (
    get_flower_client_generator(
        NETWORK_GENERATOR, FEDERATED_IID_PARTITION, lambda seq_id: list_of_ids[seq_id]
    )
)

INFO flwr 2025-03-16 22:51:10,300 | flwr_core.py:107 | cid: 1265
INFO flwr 2025-03-16 22:51:10,373 | flwr_core.py:107 | cid: 1092
INFO flwr 2025-03-16 22:51:10,377 | flwr_core.py:107 | cid: 1453
INFO flwr 2025-03-16 22:51:10,381 | flwr_core.py:107 | cid: 749
INFO flwr 2025-03-16 22:51:10,386 | flwr_core.py:107 | cid: 1169
INFO flwr 2025-03-16 22:51:10,390 | flwr_core.py:107 | cid: 1199
INFO flwr 2025-03-16 22:51:10,395 | flwr_core.py:107 | cid: 1498
INFO flwr 2025-03-16 22:51:10,398 | flwr_core.py:107 | cid: 339
INFO flwr 2025-03-16 22:51:10,402 | flwr_core.py:107 | cid: 1587
INFO flwr 2025-03-16 22:51:10,405 | flwr_core.py:107 | cid: 681
INFO flwr 2025-03-16 22:51:10,408 | flwr_core.py:107 | cid: 785
INFO flwr 2025-03-16 22:51:10,411 | flwr_core.py:107 | cid: 1300
INFO flwr 2025-03-16 22:51:10,415 | flwr_core.py:107 | cid: 736


INFO flwr 2025-03-16 22:51:10,418 | flwr_core.py:107 | cid: 630
INFO flwr 2025-03-16 22:51:10,425 | flwr_core.py:107 | cid: 801
INFO flwr 2025-03-16 22:51:10,431 | flwr_core.py:107 | cid: 1427
INFO flwr 2025-03-16 22:51:10,435 | flwr_core.py:107 | cid: 419
INFO flwr 2025-03-16 22:51:10,438 | flwr_core.py:107 | cid: 1559
INFO flwr 2025-03-16 22:51:10,442 | flwr_core.py:107 | cid: 1344
INFO flwr 2025-03-16 22:51:10,445 | flwr_core.py:107 | cid: 747
INFO flwr 2025-03-16 22:51:10,449 | flwr_core.py:107 | cid: 223
INFO flwr 2025-03-16 22:51:10,452 | flwr_core.py:107 | cid: 871
INFO flwr 2025-03-16 22:51:10,456 | flwr_core.py:107 | cid: 1300
INFO flwr 2025-03-16 22:51:10,462 | flwr_core.py:107 | cid: 816
INFO flwr 2025-03-16 22:51:10,467 | flwr_core.py:107 | cid: 133
INFO flwr 2025-03-16 22:51:10,475 | flwr_core.py:107 | cid: 1035
INFO flwr 2025-03-16 22:51:10,479 | flwr_core.py:107 | cid: 1431
INFO flwr 2025-03-16 22:51:10,483 | flwr_core.py:107 | cid: 1368
INFO flwr 2025-03-16 22:51:10,487

## **Experiments**

### LR scaling

Some rules usually are:

**Linear scaling**

$$ \text{LR} \propto \text{batch size} $$

so, starting with 0.1 at 256, we get then, 0.05 at 128, 0.025 at 64, 0.0125 at 32, 0.00625 at 16.

However, this is not always the case, and it depends on the model, the dataset, the optimizer, etc.

We can also use:

**Sqrt scaling**

$$ \text{LR} \propto \sqrt{\text{batch size}} $$

Starting with 0.1 at 256, we get then, sqrt(128/256) * 0.1 at 128, sqrt(64/256) * 0.1 at 64, sqrt(32/256) * 0.1 at 32, sqrt(16/256) * 0.1 at 16.


Or we can use **Learning Rate Finders**.

***However***, in Federated Learning usually people use lower learning rates.

While in centralized learning its normal to use LR = 0.1 to batch size = 256, in federated learning its usual to use 0.005-0.01 for batch sizes 16-32.


Their plot:

Y-axis: $\epsilon_\text{opt}(B) / \epsilon_\text{max}$, which is $\frac{1}{1+\Beta_{noise}/B}$, B batch size, $\Beta_{noise}$ is the noise scale.

X-axis: $\frac{B}{\Beta_{noise}}$, B batch size, $\Beta_{noise}$ is noise scale.

We can also observe that increasing batch size appears does in fact alter performance.

Furthermore, both $\beta_{\text{simple}}$ computation and the empirical results seem to indicate optimal batch sizes with magnitudes in the hundreds (100-1000).

In [8]:
metric_keys = ['training_time', 'samples_processed', 'noise_scale', 'train_loss', 'actual_batches']
import gc

In [9]:
B_simples = []
results = []
batch_sizes = [16, 32, 64, 128, 256]
for batch_size in batch_sizes:
    train_cfg = federated_train_config.copy()
    train_cfg["batch_size"] = batch_size
    ratio = np.sqrt(batch_size / 256)
    train_cfg["client_learning_rate"] = ratio * 0.01 # Same as centralized, but should be lower for FL

    test_cfg = federated_test_config.copy()
    test_cfg["batch_size"] = batch_size

    local_list_of_ids = sample_random_clients(num_total_clients, train_cfg["batch_size"], CID_CLIENT_GENERATOR, max_clients=max_clients)
    local_federated_client_generator = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_PARTITION, lambda seq_id: local_list_of_ids[seq_id])

    parameters_for_each_round, hist = run_simulation(
        num_rounds = 10,
        num_total_clients = num_total_clients,
        num_clients_per_round = num_clients_per_round,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = num_clients_per_round,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = local_federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        target_accuracy=0.60,
        use_target_accuracy=True
        )
    n_params = len(hist.metrics_distributed_fit.keys()) - 5
    param_keys = list(set(hist.metrics_distributed_fit.keys()) - set(metric_keys))
    hist_metrics = {key: hist.metrics_distributed_fit[key] for key in metric_keys}
    params = [hist.metrics_distributed_fit[key] for key in param_keys]
    del hist
    #gc.collect()

    res = (batch_size, parameters_for_each_round, hist_metrics, params)
    results.append(res)

INFO flwr 2025-03-16 22:51:12,501 | flwr_core.py:107 | cid: 2530


ValueError: Required files do not exist, path: /home/lcm76/L361-FL-project/femnist/client_data_mappings/fed_iid/2530/train.csv

In [ ]:
import torch
import math

B_simples = [] # 244230
n_clients = 10

K = n_clients
alpha = 0.9

for k, res in enumerate(results):
    batch_size, parameters_for_each_round, hist_metrics, params = res
    B_small = batch_size
    B_big = num_clients_per_round * batch_size
    G_local = params
    n_rounds = len(params[0])
    params_filt = [params[i] for i in range(len(params)) if len(params[i]) == n_rounds]
    G_local_by_rounds = [[params_filt[i][j][1]['all'] for i in range(len(params_filt))] for j in range(n_rounds)]
    B_simples.append([0] * n_rounds)
    for round_idx, G_local in enumerate(G_local_by_rounds):
        G_local = [[el[1] for el in G_loc] for G_loc in G_local]
        G_local_filt = [G_local[i] for i in range(len(G_local)) if len(G_local[i]) == 10]
        G_local_filt = np.array(G_local_filt)
        G_local_filt = G_local_filt.reshape(K, -1)

        G_local_filt = [torch.tensor(G_local) for G_local in G_local_filt]
        
        local_norm_squared = torch.tensor([torch.norm(G_local)**2 for G_local in G_local_filt])

        GBsmall_squared = local_norm_squared.sum() / K

        G_big = sum(G_local_filt) / K

        GBbig_squared = torch.norm(G_big)**2 

        G2 = (1 / (B_big - B_small)) * (B_big * GBbig_squared - B_small * GBsmall_squared) 

        S = (B_small * B_big / (B_big - B_small)) * (GBbig_squared - GBsmall_squared)

        B_simple = S/G2

        B_simples[k][round_idx] = B_simple

B_simples = [[el.item() for el in B_simple] for B_simple in B_simples]
print(B_simples)

### **FL run with varying batch sizes**

In [8]:
import pickle

def save_experiment(save_file_name, batch_size, parameters_for_each_round, hist):
    """Save experiment results using pickle.
    
    Args:
        save_file_name (str): Path to save the results
        batch_size (int): Batch size used in experiment
        parameters_for_each_round (list): List of model parameters for each round
        hist (History): Flower History object containing metrics
    """
    
    results_dict = {
        'batch_size': batch_size,
        'parameters_for_each_round': parameters_for_each_round,
        'history': hist
    }
    
    with open(save_file_name, 'wb') as f:  # Note: 'wb' for binary write mode
        pickle.dump(results_dict, f)

def load_experiment(file_name):
    """Load experiment results from a pickle file.
    
    Args:
        file_name (str): Path to the results file
        
    Returns:
        tuple: (batch_size, parameters_for_each_round, hist)
    """
    with open(file_name, 'rb') as f:  # Note: 'rb' for binary read mode
        results_dict = pickle.load(f)
    
    return (
        results_dict['batch_size'],
        results_dict['parameters_for_each_round'],
        results_dict['history'],
    )

total_batch_results = []

for batch_size in experiment_batch_sizes:
    train_cfg = federated_train_config.copy()
    train_cfg["batch_size"] = batch_size
    ratio = np.sqrt(batch_size / 256)
    train_cfg["client_learning_rate"] = ratio * 0.01 # Same as centralized, but should be lower for FL

    test_cfg = federated_test_config.copy()
    test_cfg["batch_size"] = batch_size

    local_list_of_ids = sample_random_clients(num_total_clients, train_cfg["batch_size"], CID_CLIENT_GENERATOR, max_clients=max_clients)
    local_federated_client_generator = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_IID_PARTITION, lambda seq_id: local_list_of_ids[seq_id])

    parameters_for_each_round, hist = run_simulation(
        num_rounds = num_rounds,
        num_total_clients = num_total_clients,
        num_clients_per_round = num_clients_per_round,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = num_clients_per_round,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = local_federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        target_accuracy=0.60,
        use_target_accuracy=True,
        )

    total_batch_results.append((batch_size, parameters_for_each_round, hist))
    save_experiment(f"results/IID_federated_local_batch_results_{batch_size}.pkl", batch_size, parameters_for_each_round, hist)

INFO flwr 2025-03-16 15:45:10,581 | flwr_core.py:107 | cid: 1265
INFO flwr 2025-03-16 15:45:10,586 | flwr_core.py:107 | cid: 1092
INFO flwr 2025-03-16 15:45:10,589 | flwr_core.py:107 | cid: 1453
INFO flwr 2025-03-16 15:45:10,593 | flwr_core.py:107 | cid: 749
INFO flwr 2025-03-16 15:45:10,596 | flwr_core.py:107 | cid: 1169
INFO flwr 2025-03-16 15:45:10,600 | flwr_core.py:107 | cid: 1199
INFO flwr 2025-03-16 15:45:10,605 | flwr_core.py:107 | cid: 1498
INFO flwr 2025-03-16 15:45:10,611 | flwr_core.py:107 | cid: 339
INFO flwr 2025-03-16 15:45:10,616 | flwr_core.py:107 | cid: 1587
INFO flwr 2025-03-16 15:45:10,622 | flwr_core.py:107 | cid: 681
INFO flwr 2025-03-16 15:45:10,626 | flwr_core.py:107 | cid: 785
INFO flwr 2025-03-16 15:45:10,629 | flwr_core.py:107 | cid: 1300
INFO flwr 2025-03-16 15:45:10,633 | flwr_core.py:107 | cid: 736
INFO flwr 2025-03-16 15:45:10,637 | flwr_core.py:107 | cid: 630
INFO flwr 2025-03-16 15:45:10,640 | flwr_core.py:107 | cid: 801
INFO flwr 2025-03-16 15:45:10,64

In [9]:
# load experiments
#batch_sizes = [16, 32, 64, 128, 256]
#total_batch_results = []
#for batch_size in batch_sizes:
#    total_batch_results.append(load_experiment(f"results/federated_batch_results_{batch_size}.pkl"))

When running FL experiments with multiple batch sizes, we do observe that there must be something as "critical" batch size.

However, we must investigate how to identify it.

## **Running FL with varying cohort sizes**

### Calculating average time per round

Since Flower is simulating multiple clients using multiple threads, a problem appears where the time we measure in multi-client setup is influenced by the context switching between threads and the waiting time.

Therefore, we derive an assumption that the time per round is constant when we fix a batch size (which we do in this experiment).

In [10]:
cohort_size = 2
train_cfg = federated_train_config.copy()
ratio = np.sqrt(cohort_size / 100)
train_cfg["client_learning_rate"] = ratio * 0.01
#train_cfg["max_batches"] = 1000

test_cfg = federated_test_config.copy()

parameters_for_each_round, hist = run_simulation(
    num_rounds = 1,
    num_total_clients = num_total_clients,
    num_clients_per_round = cohort_size,
    num_evaluate_clients = num_evaluate_clients,
    min_available_clients = num_total_clients,
    min_fit_clients = cohort_size,
    min_evaluate_clients = num_evaluate_clients,
    evaluate_fn = federated_evaluation_function,
    on_fit_config_fn = lambda _: train_cfg,
    on_evaluate_config_fn = lambda _: test_cfg,
    initial_parameters = initial_parameters,
    fit_metrics_aggregation_fn = aggregate_weighted_average,
    evaluate_metrics_aggregation_fn = aggregate_weighted_average,
    federated_client_generator = federated_client_generator,
    server_learning_rate=server_learning_rate,
    server_momentum=server_momentum,
    accept_failures=accept_failures,
    target_accuracy=0.60,
    use_target_accuracy=True,
    )

INFO flwr 2025-03-16 15:52:50,210 | experiments_simulation.py:232 | FL will execute for 1 rounds
WARNING flwr 2025-03-16 15:52:50,213 | app.py:213 | Both server and strategy were provided, ignoring strategy
INFO flwr 2025-03-16 15:52:50,213 | app.py:149 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)
INFO flwr 2025-03-16 15:52:50,215 | flwr_core.py:264 | Initializing global parameters
INFO flwr 2025-03-16 15:52:50,216 | server_returns_parameters.py:273 | Using initial parameters provided by strategy
INFO flwr 2025-03-16 15:52:50,223 | flwr_core.py:269 | Evaluating initial parameters
 11%|█         | 100/891 [00:01<00:14, 56.39it/s]
INFO flwr 2025-03-16 15:52:52,164 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 15:52:52,165 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 15:52:52,165 | server_returns_parameters.py:223 | fit_round 1: strategy

In [11]:
avg_times_per_round = []
for (round_idx, round_metrics), metrics_acc in zip(hist.metrics_distributed_fit['training_time'], hist.metrics_centralized['accuracy']):
    round_times = [t for _, t in round_metrics['all']]
    avg_time_per_round = np.mean(round_times)    
    avg_times_per_round.append(avg_time_per_round)

time_per_round = np.median(avg_times_per_round)
print(time_per_round) # 0.00427

0.004486401079165602


In [12]:
import pickle

def save_experiment(save_file_name, batch_size, parameters_for_each_round, hist):
    """Save experiment results using pickle.
    
    Args:
        save_file_name (str): Path to save the results
        batch_size (int): Batch size used in experiment
        parameters_for_each_round (list): List of model parameters for each round
        hist (History): Flower History object containing metrics
    """
    
    results_dict = {
        'batch_size': batch_size,
        'parameters_for_each_round': parameters_for_each_round,
        'history': hist
    }
    
    with open(save_file_name, 'wb') as f:  # Note: 'wb' for binary write mode
        pickle.dump(results_dict, f)

def load_experiment(file_name):
    """Load experiment results from a pickle file.
    
    Args:
        file_name (str): Path to the results file
        
    Returns:
        tuple: (batch_size, parameters_for_each_round, hist)
    """
    with open(file_name, 'rb') as f:  # Note: 'rb' for binary read mode
        results_dict = pickle.load(f)
    
    return (
        results_dict['batch_size'],
        results_dict['parameters_for_each_round'],
        results_dict['history'],
    )

total_cohort_results = []
cohort_sizes =  [5, 10, 20, 50, 75, 100]
for cohort_size in cohort_sizes:
    train_cfg = federated_train_config.copy()
    ratio = np.sqrt(cohort_size / 100)
    train_cfg["client_learning_rate"] = ratio * 0.01
    #train_cfg["max_batches"] = 1000

    test_cfg = federated_test_config.copy()

    parameters_for_each_round, hist = run_simulation(
        num_rounds = 100,
        num_total_clients = num_total_clients,
        num_clients_per_round = cohort_size,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = cohort_size,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        target_accuracy=0.60,
        use_target_accuracy=True,
        )

    total_cohort_results.append((cohort_size, parameters_for_each_round, hist))
    save_experiment(f"results/IID_federated_cohort_results_{cohort_size}.pkl", cohort_size, parameters_for_each_round=parameters_for_each_round, hist=hist)

INFO flwr 2025-03-16 15:53:31,337 | experiments_simulation.py:232 | FL will execute for 100 rounds
WARNING flwr 2025-03-16 15:53:31,339 | app.py:213 | Both server and strategy were provided, ignoring strategy
INFO flwr 2025-03-16 15:53:31,340 | app.py:149 | Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
INFO flwr 2025-03-16 15:53:31,342 | flwr_core.py:264 | Initializing global parameters
INFO flwr 2025-03-16 15:53:31,343 | server_returns_parameters.py:273 | Using initial parameters provided by strategy
INFO flwr 2025-03-16 15:53:31,344 | flwr_core.py:269 | Evaluating initial parameters
 11%|█         | 100/891 [00:01<00:12, 62.16it/s]
INFO flwr 2025-03-16 15:53:33,121 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 15:53:33,122 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 15:53:33,122 | server_returns_parameters.py:223 | fit_round 1: stra

In [13]:
#cohort_sizes =  [5, 10, 20, 50, 75, 100]
#total_cohort_results = []
#for cohort_size in cohort_sizes:
#    total_cohort_results.append(load_experiment(f"results/federated_cohort_results_{cohort_size}.pkl"))

In [14]:
for cohort_size, params, hist in total_cohort_results:
    print(f"COHORT SIZE: {cohort_size}")
    print("--------------------------------")
    for (round_idx, round_metrics), metrics_acc in zip(hist.metrics_distributed_fit['training_time'], hist.metrics_centralized['accuracy']):
        round_times = [t for _, t in round_metrics['all']]
        print(f"ROUND {round_idx}")
        print(f"TRAINING TIME: {round_times}, ACC: {metrics_acc}")

COHORT SIZE: 5
--------------------------------
ROUND 1
TRAINING TIME: [0.010376031608332899, 0.01181595227499675, 0.01129905821666739, 0.011149193416662229, 0.01078449424166763], ACC: (0, 0.0065625)
ROUND 2
TRAINING TIME: [0.012046574133328629, 0.011884521141670728, 0.011533771041663006, 0.010966275516667186, 0.011366802541672882], ACC: (1, 0.128125)
ROUND 3
TRAINING TIME: [0.011420155525001974, 0.011478981533331496, 0.012506192908329201, 0.012198787725001619, 0.012388630283328438], ACC: (2, 0.4178125)
ROUND 4
TRAINING TIME: [0.011847058883332314, 0.011380779083331542, 0.011462950133332584, 0.011688011783342252, 0.012126738533336835], ACC: (3, 0.5440625)
ROUND 5
TRAINING TIME: [0.01054851448333712, 0.010721894049992405, 0.01181397028332943, 0.011649532124997109, 0.010177518541667041], ACC: (4, 0.5709375)
COHORT SIZE: 10
--------------------------------
ROUND 1
TRAINING TIME: [0.026738469875004965, 0.026108727666670006, 0.0256221742083369, 0.024565239541662245, 0.023233930341661827, 0.

## Global batch size vs time per round

In [15]:
#16 - 256
# 5 - 100


# 80 - 25000

# 100 = 20bs * 5c
# 1000 = 50bs * 20cs
# 10000 = 200bs * 50cs
# 25000 = 250bs * 100cs

In [16]:
times_per_round = []
for batch_size in [20, 50, 200, 250]:
    cohort_size = 2
    train_cfg = federated_train_config.copy()
    ratio = np.sqrt(cohort_size / 100)
    train_cfg["client_learning_rate"] = ratio * 0.01
    #train_cfg["max_batches"] = 1000

    test_cfg = federated_test_config.copy()

    parameters_for_each_round, hist = run_simulation(
        num_rounds = 1,
        num_total_clients = num_total_clients,
        num_clients_per_round = cohort_size,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = cohort_size,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        target_accuracy=0.60,
        use_target_accuracy=True,
        )

    avg_times_per_round = []
    for (round_idx, round_metrics), metrics_acc in zip(hist.metrics_distributed_fit['training_time'], hist.metrics_centralized['accuracy']):
        round_times = [t for _, t in round_metrics['all']]
        avg_time_per_round = np.mean(round_times)    
        avg_times_per_round.append(avg_time_per_round)

    time_per_round = np.median(avg_times_per_round)
    print(time_per_round) # 0.00427
    times_per_round.append(time_per_round)

print(times_per_round) # [0.003793160191070807, 0.004481774148126263, 0.004956590107499679, 0.004278853300000662]

INFO flwr 2025-03-16 16:15:58,188 | experiments_simulation.py:232 | FL will execute for 1 rounds
WARNING flwr 2025-03-16 16:15:58,191 | app.py:213 | Both server and strategy were provided, ignoring strategy
INFO flwr 2025-03-16 16:15:58,192 | app.py:149 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)
INFO flwr 2025-03-16 16:15:58,194 | flwr_core.py:264 | Initializing global parameters
INFO flwr 2025-03-16 16:15:58,196 | server_returns_parameters.py:273 | Using initial parameters provided by strategy
INFO flwr 2025-03-16 16:15:58,200 | flwr_core.py:269 | Evaluating initial parameters
 11%|█         | 100/891 [00:02<00:16, 47.42it/s]
INFO flwr 2025-03-16 16:16:00,540 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 16:16:00,541 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 16:16:00,541 | server_returns_parameters.py:223 | fit_round 1: strategy

0.003928706841662688


 11%|█         | 100/891 [00:01<00:13, 58.31it/s]
INFO flwr 2025-03-16 16:16:46,599 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 16:16:46,601 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 16:16:46,602 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 2 clients (out of 500)
INFO flwr 2025-03-16 16:16:46,603 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 16:16:46,606 | flwr_core.py:107 | cid: 334
DEBUG flwr 2025-03-16 16:16:48,896 | server_returns_parameters.py:237 | fit_round 1 received 2 results and 0 failures
 11%|█         | 100/891 [00:00<00:05, 134.70it/s]
INFO flwr 2025-03-16 16:16:49,804 | flwr_core.py:303 | fit progress: (round 1, accuracy 0.0703125, loss 356.6226079463959, time 3.2018229140667245)
INFO flwr 2025-03-16 16:16:49,804 | server_returns_parameters.py:171 | evaluate_round 1: no clients selected, cancel
DEBUG flwr 2025-03-16 16:16:49,80

0.004374429358328295


 11%|█         | 100/891 [00:00<00:05, 138.19it/s]
INFO flwr 2025-03-16 16:17:15,619 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 16:17:15,620 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 16:17:15,620 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 2 clients (out of 500)
INFO flwr 2025-03-16 16:17:15,621 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 16:17:15,622 | flwr_core.py:107 | cid: 334
DEBUG flwr 2025-03-16 16:17:17,589 | server_returns_parameters.py:237 | fit_round 1 received 2 results and 0 failures
 11%|█         | 100/891 [00:00<00:06, 124.37it/s]
INFO flwr 2025-03-16 16:17:18,612 | flwr_core.py:303 | fit progress: (round 1, accuracy 0.0959375, loss 356.5877363681793, time 2.9921345359180123)
INFO flwr 2025-03-16 16:17:18,612 | server_returns_parameters.py:171 | evaluate_round 1: no clients selected, cancel
DEBUG flwr 2025-03-16 16:17:18,6

0.004157204416670387


 11%|█         | 100/891 [00:00<00:07, 109.60it/s]
INFO flwr 2025-03-16 16:17:49,584 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 16:17:49,585 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 16:17:49,585 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 2 clients (out of 500)
INFO flwr 2025-03-16 16:17:49,586 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 16:17:49,587 | flwr_core.py:107 | cid: 334
DEBUG flwr 2025-03-16 16:17:51,643 | server_returns_parameters.py:237 | fit_round 1 received 2 results and 0 failures
 11%|█         | 100/891 [00:00<00:06, 119.71it/s]
INFO flwr 2025-03-16 16:17:52,648 | flwr_core.py:303 | fit progress: (round 1, accuracy 0.074375, loss 354.7211527824402, time 3.0633494439534843)
INFO flwr 2025-03-16 16:17:52,649 | server_returns_parameters.py:171 | evaluate_round 1: no clients selected, cancel
DEBUG flwr 2025-03-16 16:17:52,64

0.0046010576541675625
[0.003928706841662688, 0.004374429358328295, 0.004157204416670387, 0.0046010576541675625]


In [17]:
import pickle

def save_experiment(save_file_name, batch_size, parameters_for_each_round, hist):
    """Save experiment results using pickle.
    
    Args:
        save_file_name (str): Path to save the results
        batch_size (int): Batch size used in experiment
        parameters_for_each_round (list): List of model parameters for each round
        hist (History): Flower History object containing metrics
    """
    
    results_dict = {
        'batch_size': batch_size,
        'parameters_for_each_round': parameters_for_each_round,
        'history': hist
    }
    
    with open(save_file_name, 'wb') as f:  # Note: 'wb' for binary write mode
        pickle.dump(results_dict, f)

def load_experiment(file_name):
    """Load experiment results from a pickle file.
    
    Args:
        file_name (str): Path to the results file
        
    Returns:
        tuple: (batch_size, parameters_for_each_round, hist)
    """
    with open(file_name, 'rb') as f:  # Note: 'rb' for binary read mode
        results_dict = pickle.load(f)
    
    return (
        results_dict['batch_size'],
        results_dict['parameters_for_each_round'],
        results_dict['history'],
    )

total_global_batch_results = []
cs_bs_pairs = [(5, 20), (20, 50), (50, 200), (100, 250), (100, 1000), (100, 2000), (100, 4000), (100, 12000)]
for cohort_size, batch_size in cs_bs_pairs:
    global_batch_size = batch_size * cohort_size
    train_cfg = federated_train_config.copy()
    ratio = np.sqrt(cohort_size * batch_size / 1e6)
    # ratio = 100 / 100 = 1 * 0.01
    # if i multiply by batch size, i want to divide
    print("----------------------------------------------------------", ratio * 0.01)
    train_cfg["client_learning_rate"] = ratio * 0.01
    #train_cfg["max_batches"] = 1000

    test_cfg = federated_test_config.copy()

    parameters_for_each_round, hist = run_simulation(
        num_rounds = 100,
        num_total_clients = num_total_clients,
        num_clients_per_round = cohort_size,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = cohort_size,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        target_accuracy=0.60,
        use_target_accuracy=True,
        )

    total_global_batch_results.append((global_batch_size, parameters_for_each_round, hist))
    save_experiment(f"results/IID_federated_global_batch_results_{global_batch_size}.pkl", global_batch_size, parameters_for_each_round=parameters_for_each_round, hist=hist)

INFO flwr 2025-03-16 16:18:21,457 | experiments_simulation.py:232 | FL will execute for 100 rounds
WARNING flwr 2025-03-16 16:18:21,459 | app.py:213 | Both server and strategy were provided, ignoring strategy
INFO flwr 2025-03-16 16:18:21,460 | app.py:149 | Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
INFO flwr 2025-03-16 16:18:21,462 | flwr_core.py:264 | Initializing global parameters
INFO flwr 2025-03-16 16:18:21,462 | server_returns_parameters.py:273 | Using initial parameters provided by strategy
INFO flwr 2025-03-16 16:18:21,467 | flwr_core.py:269 | Evaluating initial parameters


---------------------------------------------------------- 0.0001


 11%|█         | 100/891 [00:00<00:06, 121.63it/s]
INFO flwr 2025-03-16 16:18:22,516 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 16:18:22,516 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 16:18:22,517 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 5 clients (out of 500)
INFO flwr 2025-03-16 16:18:22,518 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 16:18:22,521 | flwr_core.py:107 | cid: 334
INFO flwr 2025-03-16 16:18:22,523 | flwr_core.py:107 | cid: 456
INFO flwr 2025-03-16 16:18:22,525 | flwr_core.py:107 | cid: 1555
INFO flwr 2025-03-16 16:18:22,528 | flwr_core.py:107 | cid: 628
DEBUG flwr 2025-03-16 16:18:29,760 | server_returns_parameters.py:237 | fit_round 1 received 5 results and 0 failures
 11%|█         | 100/891 [00:00<00:06, 131.07it/s]
INFO flwr 2025-03-16 16:18:30,729 | flwr_core.py:303 | fit progress: (round 1, accuracy 0.0709375, loss 

---------------------------------------------------------- 0.00031622776601683794


 11%|█         | 100/891 [00:01<00:09, 79.36it/s]
INFO flwr 2025-03-16 16:27:30,126 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 16:27:30,127 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 16:27:30,128 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 20 clients (out of 500)
INFO flwr 2025-03-16 16:27:30,128 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 16:27:30,129 | flwr_core.py:107 | cid: 334
INFO flwr 2025-03-16 16:27:30,130 | flwr_core.py:107 | cid: 456
INFO flwr 2025-03-16 16:27:30,132 | flwr_core.py:107 | cid: 1555
INFO flwr 2025-03-16 16:27:30,137 | flwr_core.py:107 | cid: 628
INFO flwr 2025-03-16 16:27:30,138 | flwr_core.py:107 | cid: 347
INFO flwr 2025-03-16 16:27:30,140 | flwr_core.py:107 | cid: 396
INFO flwr 2025-03-16 16:27:30,141 | flwr_core.py:107 | cid: 1017
INFO flwr 2025-03-16 16:27:30,142 | flwr_core.py:107 | cid: 1487
INFO flwr 2025-

...

 11%|█         | 100/891 [00:02<00:16, 49.09it/s]
INFO flwr 2025-03-16 17:12:04,221 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 17:12:04,221 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 17:12:04,224 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 100 clients (out of 500)
INFO flwr 2025-03-16 17:12:04,225 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 17:12:04,226 | flwr_core.py:107 | cid: 334
INFO flwr 2025-03-16 17:12:04,230 | flwr_core.py:107 | cid: 456
INFO flwr 2025-03-16 17:12:04,230 | flwr_core.py:107 | cid: 1555
INFO flwr 2025-03-16 17:12:04,231 | flwr_core.py:107 | cid: 628
INFO flwr 2025-03-16 17:12:04,232 | flwr_core.py:107 | cid: 347
INFO flwr 2025-03-16 17:12:04,234 | flwr_core.py:107 | cid: 396
INFO flwr 2025-03-16 17:12:04,236 | flwr_core.py:107 | cid: 1017
INFO flwr 2025-03-16 17:12:04,238 | flwr_core.py:107 | cid: 1487
INFO flwr 2025

---------------------------------------------------------- 0.006324555320336759


 11%|█         | 100/891 [00:02<00:17, 45.41it/s]
INFO flwr 2025-03-16 17:20:10,891 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 17:20:10,891 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 17:20:10,894 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 100 clients (out of 500)
INFO flwr 2025-03-16 17:20:10,896 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 17:20:10,899 | flwr_core.py:107 | cid: 334
INFO flwr 2025-03-16 17:20:10,900 | flwr_core.py:107 | cid: 456
INFO flwr 2025-03-16 17:20:10,901 | flwr_core.py:107 | cid: 1555
INFO flwr 2025-03-16 17:20:10,905 | flwr_core.py:107 | cid: 628
INFO flwr 2025-03-16 17:20:10,906 | flwr_core.py:107 | cid: 347
INFO flwr 2025-03-16 17:20:10,908 | flwr_core.py:107 | cid: 396
INFO flwr 2025-03-16 17:20:10,910 | flwr_core.py:107 | cid: 1017
INFO flwr 2025-03-16 17:20:10,914 | flwr_core.py:107 | cid: 1487
INFO flwr 2025

---------------------------------------------------------- 0.01095445115010332


 11%|█         | 100/891 [00:01<00:13, 56.86it/s]
INFO flwr 2025-03-16 17:28:36,821 | flwr_core.py:272 | initial parameters (loss, other metrics): 413.6843070983887, {'accuracy': 0.0065625}
INFO flwr 2025-03-16 17:28:36,822 | flwr_core.py:280 | FL starting - Target accuracy: 0.6
DEBUG flwr 2025-03-16 17:28:36,825 | server_returns_parameters.py:223 | fit_round 1: strategy sampled 100 clients (out of 500)
INFO flwr 2025-03-16 17:28:36,825 | flwr_core.py:107 | cid: 851
INFO flwr 2025-03-16 17:28:36,827 | flwr_core.py:107 | cid: 334
INFO flwr 2025-03-16 17:28:36,830 | flwr_core.py:107 | cid: 456
INFO flwr 2025-03-16 17:28:36,832 | flwr_core.py:107 | cid: 1555
INFO flwr 2025-03-16 17:28:36,834 | flwr_core.py:107 | cid: 628
INFO flwr 2025-03-16 17:28:36,837 | flwr_core.py:107 | cid: 347
INFO flwr 2025-03-16 17:28:36,839 | flwr_core.py:107 | cid: 396
INFO flwr 2025-03-16 17:28:36,843 | flwr_core.py:107 | cid: 1017
INFO flwr 2025-03-16 17:28:36,845 | flwr_core.py:107 | cid: 1487
INFO flwr 2025